In [8]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df_processed = pd.read_csv('../DATASET/DATA[T].csv')

df_original = pd.read_csv('../DATASET/DATA.csv')

In [10]:
X_features = df_processed.drop('Churn', axis=1)
similarity_matrix = cosine_similarity(X_features)
similarity_df = pd.DataFrame(similarity_matrix, index=df_original['customerID'], columns=df_original['customerID'])

In [ ]:
def get_service_recommendations(customer_id, num_recommendations=3):
    """
    Generates service recommendations for a given customer based on finding similar users.
    """
    similar_customers = similarity_df[customer_id].sort_values(ascending=False).iloc[1:11]
    similar_customer_services = df_original[df_original['customerID'].isin(similar_customers.index)]
    target_customer_services = df_original[df_original['customerID'] == customer_id]

    
    recommendations = {}
    service_columns = ['PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

    for service in service_columns:
        if target_customer_services[service].iloc[0] == 'No':
            recommendation_score = similar_customer_services[service].value_counts().get('Yes', 0)
            if recommendation_score > 0:
                recommendations[service] = recommendation_score

    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return dict(sorted_recommendations[:num_recommendations])

In [27]:
sample_customer_id = '8091-TTVAX'

recommended_services = get_service_recommendations(sample_customer_id)

print("\n" + "="*50)
if recommended_services:
    print(f"TOP RECOMMENDATIONS FOR CUSTOMER {sample_customer_id}:")
    for service, score in recommended_services.items():
        print(f"- Recommend '{service}' (Subscribed to by {score} out of 10 similar customers)")
else:
    print(f"No new service recommendations for customer {sample_customer_id}.")
print("="*50)


TOP RECOMMENDATIONS FOR CUSTOMER 8091-TTVAX:
- Recommend 'OnlineBackup' (Subscribed to by 4 out of 10 similar customers)
- Recommend 'OnlineSecurity' (Subscribed to by 1 out of 10 similar customers)


In [28]:
def assign_and_recommend_plan(customer_id):
    """
    First, finds potential service recommendations for a user.
    Then, uses the number of recommendations to assign them to a logical plan tier.
    """
    service_recs = get_service_recommendations(customer_id, num_recommendations=10)
    recommendation_count = len(service_recs)
    
    assigned_plan = ''
    if recommendation_count <= 1:
        assigned_plan = 'Ultra'
        reason = f"You are a power user with high engagement (only {recommendation_count} upsell opportunities found)."
    elif recommendation_count >= 4:
        assigned_plan = 'Basic'
        reason = f"You have a foundational set of services with lots of potential for new features ({recommendation_count} upsell opportunities found)."
    else:
        assigned_plan = 'Plus'
        reason = f"You have a solid set of services with some room to grow ({recommendation_count} upsell opportunities found)."
        
    # Step 3: Return a clear recommendation
    print(f"PLAN ANALYSIS FOR CUSTOMER {customer_id}:")
    print(f"- Potential service recommendations: {recommendation_count}")
    print(f"- Logically assigned plan: '{assigned_plan}'")
    print(f"- Reason: {reason}")

In [29]:
customer_basic = '8091-TTVAX'
assign_and_recommend_plan(customer_basic)

print("\n" + "-"*50 + "\n")

customer_ultra = '7590-VHVEG'
assign_and_recommend_plan(customer_ultra)

PLAN ANALYSIS FOR CUSTOMER 8091-TTVAX:
- Potential service recommendations: 2
- Logically assigned plan: 'Plus'
- Reason: You have a solid set of services with some room to grow (2 upsell opportunities found).

--------------------------------------------------

PLAN ANALYSIS FOR CUSTOMER 7590-VHVEG:
- Potential service recommendations: 1
- Logically assigned plan: 'Ultra'
- Reason: You are a power user with high engagement (only 1 upsell opportunities found).
